# 🧬 Implementasi Algoritma Genetika untuk Penentuan Kelompok KKM Reguler
## UIN Malang

---

**Tujuan**: Mengelompokkan mahasiswa ke dalam kelompok-kelompok KKM Reguler yang optimal dengan mempertimbangkan:
- Keberadaan anggota HTQ
- Heterogenitas jurusan
- Proporsi jenis kelamin
- Jumlah anggota per kelompok

**Metode**: Genetic Algorithm dengan PMX Crossover dan Reciprocal Exchange Mutation

## 1. Import Required Libraries

In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
from datetime import datetime
import os

# Set random seed for reproducibility
random.seed(42)
np.random.seed(42)

print("✅ Libraries imported successfully!")
print(f"Pandas version: {pd.__version__}")
print(f"Numpy version: {np.__version__}")

✅ Libraries imported successfully!
Pandas version: 2.3.3
Numpy version: 2.3.4


## 2. Load and Validate Data

In [2]:
# Load data
csv_path = '../data/master_data.csv'
df = pd.read_csv(csv_path)

# Validate required columns
required_cols = ['ID', 'Jenis Kelamin', 'Jurusan', 'HTQ']
assert all(col in df.columns for col in required_cols), "Missing required columns!"

# Check missing values
assert df[required_cols].isnull().sum().sum() == 0, "Missing values detected!"

# Check duplicate IDs
assert df['ID'].duplicated().sum() == 0, "Duplicate IDs detected!"

print("="*80)
print("DATA VALIDATION PASSED ✅")
print("="*80)
print(f"\nTotal Mahasiswa: {len(df)}")
print(f"Jumlah Jurusan: {df['Jurusan'].nunique()}")
print(f"\nDistribusi Jenis Kelamin:")
print(df['Jenis Kelamin'].value_counts())
print(f"\nDistribusi HTQ:")
print(df['HTQ'].value_counts())
print(f"\nSample Data:")
display(df.head(10))

DATA VALIDATION PASSED ✅

Total Mahasiswa: 2338
Jumlah Jurusan: 24

Distribusi Jenis Kelamin:
Jenis Kelamin
PR    1391
LK     947
Name: count, dtype: int64

Distribusi HTQ:
HTQ
Tidak    2112
Ya        226
Name: count, dtype: int64

Sample Data:


,ID,Jenis Kelamin,Jurusan,HTQ
0,1,PR,BAHASA DAN SASTRA INGGRIS,Tidak
1,2,PR,BIOLOGI,Tidak
2,3,PR,TEKNIK INFORMATIKA,Tidak
3,4,LK,BAHASA DAN SASTRA ARAB,Tidak
4,5,LK,ILMU AL-QUR`AN DAN TAFSIR,Tidak
5,6,PR,AL-AHWAL AL-SYAKHSHIYYAH,Tidak
6,7,PR,PSIKOLOGI,Tidak
7,8,PR,PERBANKAN SYARI`AH,Tidak
8,9,LK,HUKUM BISNIS SYARI'AH,Tidak
9,10,LK,MANAJEMEN,Tidak


## 3. Data Preprocessing

In [ ]:
def preprocess_data(df, jumlah_kelompok):
    """
    Preprocess data dan hitung semua statistik yang diperlukan
    """
    df_clean = df.copy()
    
    # Normalize HTQ to binary
    df_clean['HTQ'] = df_clean['HTQ'].apply(lambda x: 1 if str(x).lower() in ['ya', 'lulus', '1'] else 0)
    
    # Calculate aggregate statistics
    N = len(df_clean)
    L = (df_clean['Jenis Kelamin'] == 'LK').sum()
    P = (df_clean['Jenis Kelamin'] == 'PR').sum()
    K = jumlah_kelompok
    
    # Calculate expected proportions
    PL = L / N
    PP = P / N
    
    # Calculate expected sizes per group
    A = N // K
    sisa = N % K
    
    expected_sizes = []
    for i in range(K):
        if i < sisa:
            expected_sizes.append(A + 1)
        else:
            expected_sizes.append(A)
    
    # Max fitness
    max_fitness = K * 4
    
    preprocessed = {
        'df_clean': df_clean,
        'N': N,
        'L': L,
        'P': P,
        'K': K,
        'PL': PL,
        'PP': PP,
        'A': A,
        'sisa': sisa,
        'expected_sizes': expected_sizes,
        'max_fitness': max_fitness
    }
    
    return preprocessed

# Parameter GA
JUMLAH_KELOMPOK = 190

# Preprocess
preprocessed = preprocess_data(df, JUMLAH_KELOMPOK)
df_clean = preprocessed['df_clean']

print("="*80)
print("PREPROCESSING COMPLETE ✅")
print("="*80)
print(f"\nTotal Mahasiswa (N): {preprocessed['N']}")
print(f"Laki-laki (L): {preprocessed['L']}")
print(f"Perempuan (P): {preprocessed['P']}")
print(f"Jumlah Kelompok (K): {preprocessed['K']}")
print(f"\nProporsi Ideal:")
print(f"  PL (Laki-laki): {preprocessed['PL']:.4f} ({preprocessed['PL']:.2%})")
print(f"  PP (Perempuan): {preprocessed['PP']:.4f} ({preprocessed['PP']:.2%})")
print(f"\nUkuran Kelompok:")
print(f"  Base size (A): {preprocessed['A']}")
print(f"  Sisa: {preprocessed['sisa']}")
print(f"  Expected sizes: {preprocessed['expected_sizes'][:5]}... (showing first 5)")
print(f"\nMax Fitness: {preprocessed['max_fitness']}")

PREPROCESSING COMPLETE ✅

Total Mahasiswa (N): 2338
Laki-laki (L): 947
Perempuan (P): 1391
Jumlah Kelompok (K): 190

Proporsi Ideal:
  PL (Laki-laki): 0.4050 (40.50%)
  PP (Perempuan): 0.5950 (59.50%)

Ukuran Kelompok:
  Base size (A): 12
  Sisa: 58
  Expected sizes: [13, 13, 13, 13, 13]... (showing first 5)

Max Fitness: 760


## 4. Constraint Evaluation Functions

In [ ]:
def evaluate_C1(kelompok_ids, df_clean):
    """C1: Minimal 1 anggota HTQ per kelompok"""
    kelompok_df = df_clean[df_clean['ID'].isin(kelompok_ids)]
    htq_count = kelompok_df['HTQ'].sum()
    return 1 if htq_count >= 1 else 0

def evaluate_C2(kelompok_ids, df_clean):
    """C2: Jurusan unik > 50% dari jumlah anggota"""
    kelompok_df = df_clean[df_clean['ID'].isin(kelompok_ids)]
    A = len(kelompok_ids)
    batas = 0.5 * A
    jurusan_unique = kelompok_df['Jurusan'].nunique()
    return 1 if jurusan_unique > batas else 0

def evaluate_C3(kelompok_ids, df_clean, preprocessed):
    """C3: Proporsi gender dalam toleransi ±10%"""
    kelompok_df = df_clean[df_clean['ID'].isin(kelompok_ids)]
    
    actual_L = (kelompok_df['Jenis Kelamin'] == 'LK').sum()
    actual_P = (kelompok_df['Jenis Kelamin'] == 'PR').sum()
    total = len(kelompok_ids)
    
    if total == 0:
        return 0
    
    actual_PL = actual_L / total
    actual_PP = actual_P / total
    
    PL = preprocessed['PL']
    PP = preprocessed['PP']
    
    toleransi = 0.1
    min_L = PL - toleransi
    max_L = PL + toleransi
    min_P = PP - toleransi
    max_P = PP + toleransi
    
    kondisi_L = (min_L <= actual_PL <= max_L)
    kondisi_P = (min_P <= actual_PP <= max_P)
    
    return 1 if (kondisi_L and kondisi_P) else 0

def evaluate_C4(kelompok_ids, kelompok_index, preprocessed):
    """C4: Jumlah anggota sesuai expected size"""
    actual_size = len(kelompok_ids)
    expected_size = preprocessed['expected_sizes'][kelompok_index]
    return 1 if actual_size == expected_size else 0

print("✅ Constraint evaluation functions defined!")
print("\n📋 Constraint Definitions:")
print("  C1: Minimal 1 anggota HTQ per kelompok")
print("  C2: Jurusan unik > 50% dari jumlah anggota")
print("  C3: Proporsi gender dalam toleransi ±10%")
print("  C4: Jumlah anggota sesuai expected size")

✅ Constraint evaluation functions defined!

📋 Constraint Definitions:
  C1: Minimal 1 anggota HTQ per kelompok
  C2: Jurusan unik > 50% dari jumlah anggota
  C3: Proporsi gender dalam toleransi ±10%
  C4: Jumlah anggota sesuai expected size


## 5. Fitness Calculation & Decode Kromosom

In [ ]:
def decode_kromosom(kromosom, expected_sizes):
    """Memotong kromosom menjadi kelompok berdasarkan expected_sizes"""
    kelompok_list = []
    start_idx = 0
    
    for size in expected_sizes:
        kelompok = kromosom[start_idx : start_idx + size]
        kelompok_list.append(kelompok)
        start_idx += size
    
    return kelompok_list

def calculate_fitness(kromosom, df_clean, preprocessed):
    """Hitung total fitness dari semua constraint"""
    expected_sizes = preprocessed['expected_sizes']
    kelompok_list = decode_kromosom(kromosom, expected_sizes)
    
    total_fitness = 0
    
    for g, kelompok_ids in enumerate(kelompok_list):
        C1 = evaluate_C1(kelompok_ids, df_clean)
        C2 = evaluate_C2(kelompok_ids, df_clean)
        C3 = evaluate_C3(kelompok_ids, df_clean, preprocessed)
        C4 = evaluate_C4(kelompok_ids, g, preprocessed)
        
        total_fitness += (C1 + C2 + C3 + C4)
    
    return total_fitness

def calculate_normalized_fitness(kromosom, df_clean, preprocessed):
    """Hitung normalized fitness [0.0, 1.0]"""
    raw_fitness = calculate_fitness(kromosom, df_clean, preprocessed)
    max_fitness = preprocessed['max_fitness']
    return raw_fitness / max_fitness

print("✅ Fitness calculation functions defined!")

✅ Fitness calculation functions defined!


## 6. Population Initialization

In [ ]:
def initialize_population(mahasiswa_ids, popsize):
    """Generate PopSize kromosom secara random shuffle"""
    population = []
    
    for _ in range(popsize):
        kromosom = mahasiswa_ids.copy()
        random.shuffle(kromosom)
        population.append(kromosom)
    
    return population

print("✅ Population initialization function defined!")

✅ Population initialization function defined!


## 7. Parent Selection Functions

In [ ]:
def select_parents_for_crossover(population, num_offspring):
    """Pilih parent untuk crossover secara random (Parent1 ≠ Parent2)"""
    if num_offspring % 2 == 0:
        num_crossovers = num_offspring // 2
        take_single = False
    else:
        num_crossovers = (num_offspring // 2) + 1
        take_single = True
    
    parent_pairs = []
    for _ in range(num_crossovers):
        parent1, parent2 = random.sample(population, 2)
        parent_pairs.append((parent1, parent2))
    
    return parent_pairs, take_single

def select_parents_for_mutation(population, num_offspring):
    """Pilih parent untuk mutation secara random (with replacement)"""
    selected_parents = []
    
    for _ in range(num_offspring):
        parent = random.choice(population)
        selected_parents.append(parent)
    
    return selected_parents

print("✅ Parent selection functions defined!")

✅ Parent selection functions defined!


## 8. PMX Crossover Implementation

In [ ]:
def pmx_crossover(parent1, parent2):
    """Partially Matched Crossover (PMX)"""
    length = len(parent1)
    
    # Step 1: Pilih titik potong
    point1, point2 = sorted(random.sample(range(length), 2))
    
    # Step 2: Buat primitive offspring
    offspring1 = parent1.copy()
    offspring2 = parent2.copy()
    
    offspring1[point1:point2+1] = parent2[point1:point2+1]
    offspring2[point1:point2+1] = parent1[point1:point2+1]
    
    # Step 3: Buat mapping
    mapping = {}
    for i in range(point1, point2 + 1):
        gene_p1 = parent1[i]
        gene_p2 = parent2[i]
        mapping[gene_p1] = gene_p2
        mapping[gene_p2] = gene_p1
    
    # Step 4: Resolve konflik
    def resolve(offspring):
        for i in range(length):
            if point1 <= i <= point2:
                continue
            
            gene = offspring[i]
            while gene in offspring[point1:point2+1]:
                gene = mapping.get(gene, gene)
            
            offspring[i] = gene
        
        return offspring
    
    offspring1 = resolve(offspring1)
    offspring2 = resolve(offspring2)
    
    return offspring1, offspring2

def apply_crossover(crossover_parents, take_single):
    """Apply PMX untuk semua parent pairs"""
    crossover_offspring = []
    
    for i, (parent1, parent2) in enumerate(crossover_parents):
        off1, off2 = pmx_crossover(parent1, parent2)
        
        if take_single and i == len(crossover_parents) - 1:
            selected_offspring = random.choice([off1, off2])
            crossover_offspring.append(selected_offspring)
        else:
            crossover_offspring.extend([off1, off2])
    
    return crossover_offspring

print("✅ PMX Crossover functions defined!")

✅ PMX Crossover functions defined!


## 9. Reciprocal Exchange Mutation

In [9]:
def reciprocal_exchange_mutation(parent):
    """Reciprocal Exchange Mutation - tukar 2 gen secara random"""
    offspring = parent.copy()
    
    length = len(offspring)
    pivot1, pivot2 = random.sample(range(length), 2)
    
    offspring[pivot1], offspring[pivot2] = offspring[pivot2], offspring[pivot1]
    
    return offspring

def apply_mutation(mutation_parents):
    """Apply Reciprocal Exchange untuk semua mutation parents"""
    mutation_offspring = []
    
    for parent in mutation_parents:
        offspring = reciprocal_exchange_mutation(parent)
        mutation_offspring.append(offspring)
    
    return mutation_offspring

print("✅ Reciprocal Exchange Mutation functions defined!")

✅ Reciprocal Exchange Mutation functions defined!


## 10. Elitism Replacement

In [ ]:
def elitism_replacement(current_population, crossover_offspring, 
                        mutation_offspring, df_clean, preprocessed):
    """Elitism Replacement Strategy"""
    PopSize = len(current_population)
    
    # Step 1: Gabungkan semua kandidat
    all_candidates = current_population + crossover_offspring + mutation_offspring
    
    # Step 2: Hitung fitness untuk semua
    candidates_with_fitness = []
    for kromosom in all_candidates:
        fitness = calculate_fitness(kromosom, df_clean, preprocessed)
        normalized_fitness = fitness / preprocessed['max_fitness']
        candidates_with_fitness.append((kromosom, fitness, normalized_fitness))
    
    # Step 3: Sort descending
    candidates_with_fitness.sort(key=lambda x: x[1], reverse=True)
    
    # Step 4: Ambil PopSize terbaik
    next_population = []
    for i in range(PopSize):
        kromosom, fitness, norm_fitness = candidates_with_fitness[i]
        next_population.append(kromosom)
    
    # Best solution
    best_kromosom, best_fitness, best_norm_fitness = candidates_with_fitness[0]
    
    return next_population, best_fitness, best_norm_fitness, best_kromosom

print("✅ Elitism Replacement function defined!")

✅ Elitism Replacement function defined!


## 11. Main Genetic Algorithm Loop

In [ ]:
# Parameter Algoritma Genetika
params = {
    'jumlah_kelompok': JUMLAH_KELOMPOK,
    'popsize': 10,
    'cr': 0.1,
    'mr': 0.1,
    'max_generation': 10,
    'target_fitness': 0.8
}

print("="*80)
print("GENETIC ALGORITHM PARAMETERS")
print("="*80)
for key, value in params.items():
    print(f"{key:20s}: {value}")
print("="*80)

GENETIC ALGORITHM PARAMETERS
jumlah_kelompok     : 190
popsize             : 10
cr                  : 0.1
mr                  : 0.1
max_generation      : 10
target_fitness      : 0.8


In [ ]:
# Initialize Population
mahasiswa_ids = df_clean['ID'].tolist()
population = initialize_population(mahasiswa_ids, params['popsize'])

print(f"✅ Initial population created: {len(population)} individuals")
print(f"   Each chromosome length: {len(population[0])}")

# Initialize tracking variables
generation = 0
history = []
global_best_kromosom = None
global_best_fitness = 0
global_best_norm_fitness = 0.0

print("\n🚀 Starting Genetic Algorithm...")
print("="*80)

✅ Initial population created: 10 individuals
   Each chromosome length: 2338

🚀 Starting Genetic Algorithm...


: 

In [ ]:
# Main GA Loop dengan Progress Tracking
import time

start_time = time.time()
print(f"⏱️  Start time: {datetime.now().strftime('%Y-%m-%d %H:%M:%S')}")
print("="*80)

while True:
    generation += 1
    
    # CROSSOVER
    num_crossover_offspring = int(params['popsize'] * params['cr'])
    crossover_parents, take_single = select_parents_for_crossover(
        population, num_crossover_offspring
    )
    crossover_offspring = apply_crossover(crossover_parents, take_single)
    
    # MUTATION
    num_mutation_offspring = int(params['popsize'] * params['mr'])
    mutation_parents = select_parents_for_mutation(population, num_mutation_offspring)
    mutation_offspring = apply_mutation(mutation_parents)
    
    # REPLACEMENT (ELITISM)
    next_population, best_fitness, best_norm_fitness, best_kromosom = \
        elitism_replacement(population, crossover_offspring, mutation_offspring,
                          df_clean, preprocessed)
    
    # Update global best
    if best_fitness > global_best_fitness:
        global_best_kromosom = best_kromosom
        global_best_fitness = best_fitness
        global_best_norm_fitness = best_norm_fitness
        print(f"🆕 New best found at Gen {generation}: {best_norm_fitness:.4f}")
    
    # Track history
    history.append({
        'generation': generation,
        'best_fitness': best_fitness,
        'best_norm_fitness': best_norm_fitness
    })
    
    # Print progress SETIAP GENERASI untuk monitoring
    if generation % 5 == 0 or generation == 1:  # Print setiap 5 generasi
        elapsed = time.time() - start_time
        print(f"Gen {generation:4d} | Fit: {best_fitness:3d}/{preprocessed['max_fitness']} " +
              f"| Norm: {best_norm_fitness:.4f} ({best_norm_fitness:6.2%}) " +
              f"| Time: {elapsed/60:.2f}m")
    
    # TERMINATION CHECK
    if best_norm_fitness >= params['target_fitness']:
        print(f"\n🎯 Target fitness tercapai: {best_norm_fitness:.4f} >= {params['target_fitness']:.4f}")
        break
    
    if generation >= params['max_generation']:
        print(f"\n⏱️ Generasi maksimal tercapai: {generation} >= {params['max_generation']}")
        break
    
    # Update population
    population = next_population

elapsed_time = time.time() - start_time
print("="*80)
print(f"✅ Genetic Algorithm COMPLETED!")
print(f"   Total generations: {generation}")
print(f"   Elapsed time: {elapsed_time/60:.2f} minutes")
print(f"   Final fitness: {global_best_norm_fitness:.4f}")
print("="*80)

⏱️  Start time: 2025-11-06 08:32:39


## 12. Results Visualization & Analysis

In [ ]:
# Decode best solution
kelompok_list = decode_kromosom(global_best_kromosom, preprocessed['expected_sizes'])

print("="*80)
print("BEST SOLUTION FOUND")
print("="*80)
print(f"\nFitness: {global_best_fitness}/{preprocessed['max_fitness']}")
print(f"Normalized: {global_best_norm_fitness:.4f} ({global_best_norm_fitness:.2%})")
print(f"Total Generations: {generation}")
print(f"\nJumlah Kelompok: {len(kelompok_list)}")

# Detailed constraint analysis
print("\n" + "="*80)
print("CONSTRAINT SATISFACTION ANALYSIS")
print("="*80)

constraint_summary = {
    'C1': 0,  # HTQ
    'C2': 0,  # Jurusan diversity
    'C3': 0,  # Gender proportion
    'C4': 0   # Group size
}

for i, kelompok_ids in enumerate(kelompok_list, 1):
    C1 = evaluate_C1(kelompok_ids, df_clean)
    C2 = evaluate_C2(kelompok_ids, df_clean)
    C3 = evaluate_C3(kelompok_ids, df_clean, preprocessed)
    C4 = evaluate_C4(kelompok_ids, i-1, preprocessed)
    
    constraint_summary['C1'] += C1
    constraint_summary['C2'] += C2
    constraint_summary['C3'] += C3
    constraint_summary['C4'] += C4

print(f"\nConstraint Satisfaction Rate:")
print(f"  C1 (HTQ):            {constraint_summary['C1']}/{preprocessed['K']} ({constraint_summary['C1']/preprocessed['K']:.2%})")
print(f"  C2 (Jurusan):        {constraint_summary['C2']}/{preprocessed['K']} ({constraint_summary['C2']/preprocessed['K']:.2%})")
print(f"  C3 (Gender):         {constraint_summary['C3']}/{preprocessed['K']} ({constraint_summary['C3']/preprocessed['K']:.2%})")
print(f"  C4 (Group Size):     {constraint_summary['C4']}/{preprocessed['K']} ({constraint_summary['C4']/preprocessed['K']:.2%})")

In [ ]:
# Plot Convergence Curve
plt.figure(figsize=(12, 6))

generations = [h['generation'] for h in history]
best_fitness_norm = [h['best_norm_fitness'] for h in history]

plt.plot(generations, best_fitness_norm, 'b-', linewidth=2, label='Best Fitness')
plt.axhline(y=params['target_fitness'], color='r', linestyle='--', 
            linewidth=1, label=f"Target ({params['target_fitness']:.2f})")
plt.xlabel('Generation', fontsize=12)
plt.ylabel('Normalized Fitness', fontsize=12)
plt.title('Genetic Algorithm Convergence Curve', fontsize=14, fontweight='bold')
plt.grid(True, alpha=0.3)
plt.legend(fontsize=10)
plt.tight_layout()

# Save plot
output_dir = '../output'
os.makedirs(output_dir, exist_ok=True)
timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
plot_filename = f'{output_dir}/convergence_{timestamp}.png'
plt.savefig(plot_filename, dpi=300, bbox_inches='tight')
print(f"\n📊 Convergence plot saved to: {plot_filename}")

plt.show()

## 13. Export Results to Excel

In [ ]:
# Prepare data for export
kelompok_data = []

for i, kelompok_ids in enumerate(kelompok_list, 1):
    kelompok_df = df_clean[df_clean['ID'].isin(kelompok_ids)]
    
    # Calculate constraints for this group
    C1 = evaluate_C1(kelompok_ids, df_clean)
    C2 = evaluate_C2(kelompok_ids, df_clean)
    C3 = evaluate_C3(kelompok_ids, df_clean, preprocessed)
    C4 = evaluate_C4(kelompok_ids, i-1, preprocessed)
    
    for _, row in kelompok_df.iterrows():
        kelompok_data.append({
            'Kelompok': i,
            'ID_Mahasiswa': row['ID'],
            'Nama': f"Mahasiswa_{row['ID']}",  # Placeholder for nama
            'Jenis_Kelamin': row['Jenis Kelamin'],
            'Jurusan': row['Jurusan'],
            'HTQ': 'Ya' if row['HTQ'] == 1 else 'Tidak',
            'C1_HTQ': C1,
            'C2_Jurusan': C2,
            'C3_Gender': C3,
            'C4_Size': C4,
            'Total_Score': C1 + C2 + C3 + C4
        })

# Create DataFrame
df_result = pd.DataFrame(kelompok_data)

# Export to Excel
excel_filename = f'{output_dir}/hasil_kelompok_kkm_{timestamp}.xlsx'
df_result.to_excel(excel_filename, index=False, sheet_name='Hasil Kelompok')

print(f"📄 Results exported to: {excel_filename}")
print(f"   Total rows: {len(df_result)}")
print(f"   Total groups: {df_result['Kelompok'].nunique()}")

# Display sample
print("\n📋 Sample of exported data:")
display(df_result.head(15))

In [ ]:
# Summary Statistics per Group
print("="*80)
print("GROUP SUMMARY STATISTICS")
print("="*80)

group_summary = df_result.groupby('Kelompok').agg({
    'ID_Mahasiswa': 'count',
    'HTQ': lambda x: (x == 'Ya').sum(),
    'Jurusan': 'nunique',
    'Total_Score': 'first'
}).rename(columns={
    'ID_Mahasiswa': 'Jumlah_Anggota',
    'HTQ': 'Jumlah_HTQ',
    'Jurusan': 'Jurusan_Unik'
})

# Add gender distribution
gender_dist = df_result.groupby('Kelompok')['Jenis_Kelamin'].value_counts().unstack(fill_value=0)
if 'LK' in gender_dist.columns:
    group_summary['LK'] = gender_dist['LK']
if 'PR' in gender_dist.columns:
    group_summary['PR'] = gender_dist['PR']

display(group_summary.head(10))

print(f"\n📊 Statistics Summary:")
print(f"   Average group size: {group_summary['Jumlah_Anggota'].mean():.2f}")
print(f"   Min group size: {group_summary['Jumlah_Anggota'].min()}")
print(f"   Max group size: {group_summary['Jumlah_Anggota'].max()}")
print(f"   Average HTQ per group: {group_summary['Jumlah_HTQ'].mean():.2f}")
print(f"   Average unique majors per group: {group_summary['Jurusan_Unik'].mean():.2f}")

## 🎉 Implementation Complete!

### Summary

Implementasi Algoritma Genetika untuk penentuan kelompok KKM telah selesai dengan hasil:

**Key Features:**
- ✅ Data preprocessing & validation
- ✅ 4 Constraint evaluation (C1-C4)
- ✅ PMX Crossover implementation
- ✅ Reciprocal Exchange Mutation
- ✅ Elitism Replacement Strategy
- ✅ Convergence tracking & visualization
- ✅ Excel export with detailed analysis

**Next Steps:**
1. Review hasil pengelompokan di file Excel
2. Analisis constraint satisfaction rate
3. Tune parameter GA jika diperlukan (popsize, cr, mr)
4. Run ulang dengan parameter berbeda untuk comparison